In [2]:
# load data from csv
# path for input and target data tables

diagnosis_path = 'data/diagnosis_hadm.csv'
discharge_path ='data/discharge.csv'
edstays_path = 'data/edstays.csv'
radiology_path = 'data/radiology.csv'
triage_path = 'data/triage.csv'
target_path = 'data/discharge_target.csv'
discharge_sections_path = 'data/discharge_sections.csv'
radiology_sections_path = 'data/radiology_sections.csv'

import pandas as pd
import re
import os

# read data
diagnosis_df = pd.read_csv(diagnosis_path, keep_default_na=False)
discharge_df = pd.read_csv(discharge_path, keep_default_na=False)
edstays_df = pd.read_csv(edstays_path, keep_default_na=False)
radiology_df = pd.read_csv(radiology_path, keep_default_na=False)
triage_df = pd.read_csv(triage_path, keep_default_na=False)
target_df = pd.read_csv(target_path, keep_default_na=False)

discharge_sections_df = pd.read_csv(discharge_sections_path, keep_default_na=False)
radiology_sections_df = pd.read_csv(radiology_sections_path, keep_default_na=False)

In [3]:
target_df.head()

,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count
0,10001884-DS-35,24962904,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",Ms. ___ is a ___ female with history of \nCOPD...,87,358
1,10003019-DS-22,22774359,"Dear Mr. ___,\n\nIt has been our pleasure to b...",___ male with h/o Hodgkin's lymphoma C1D17 ABV...,140,78
2,10003299-DS-7,29323205,Dear ___ were hospitalized due to symptoms of ...,___ RH female with a PMHx of paramedian pontin...,242,386
3,10003502-DS-7,20459702,It was a pleasure caring for you at ___ \n___....,Hospitalization Summary: \nMs. ___ is an ___ y...,74,50
4,10004322-DS-21,28755331,"Dear Mr. ___,\n\nIt was our pleasure to care f...",___ with h/o psychosis admitted because of mul...,69,287


In [21]:
for patient_id in target_df['hadm_id'][:5]:
    brief_hospital_course_prompt = generate_bhc(patient_id)
    discharge_instructions_prompt = generate_discharge_instructions(patient_id)
    bhc_summary = target_df[target_df.hadm_id==patient_id]['brief_hospital_course'][0]
    discharge_summary = target_df[target_df.hadm_id==patient_id]['discharge_instructions'][0]

In [19]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Topic:
{instruction}

### Input:
{input}
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
{instruction}

### Response:
"""

In [5]:
def generate_with_model(instruction, context_input):
    # Generate the prompt using the instruction and context
    prompt = generate_prompt(instruction, context_input)
    return prompt

In [6]:
# extract HPI summary USE THIS
def generate_bhc(hadm_id):
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with text and data from a specific patient's medical records and clinical notes
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    Answer clearly and concisely.
    '''.replace('\n', '')
    
    # prompt = '''
    # Using the following patient specific data, 
    # briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
    # Then, create a list of the patient's acute and chronic medical conditions and the associated treatments course.
    # '''.replace('\n', '')
    prompt = '''
    Using the following patient specific data, 
    briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.
    '''.replace('\n', '')
    
    context_str =''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['CC']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The chief complaint was - {context_data} \n"
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The History of present illness: \n{context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission and discharge medication lists:\n{context_data} \n"
    # cols = ['Pertinent Results']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Pertinent Results were:\n{context_data} \n"
    cols = ['Major Surgical Procedure']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The major procedures were:\n{context_data} \n"
    cols = ['Discharge Diagnosis']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The patient was diagnosed with:\n{context_data} \n"
    
    df = diagnosis_df
    cols = ['icd_title']
    cur_df = df[df['hadm_id'] == hadm_id]
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
    instruction = f"{topic} \n Instruction: {prompt}"
    context_input = f"<text> Text:{context_str}</text> "
    # print(f"Context for BHC: {context_input}")
    response = generate_with_model(instruction, context_input)
    return response

In [7]:

def generate_discharge_instructions(hadm_id):
    topic = '''
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR) 
    from a patient's hospital admission. 
    You will be provided with text and data from a specific patient's medical records and clinical notes
    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.  
    Answer clearly and concisely.
    '''.replace('\n', '')
    
    prompt = '''
    Given the following information from a patient's hospital stay,
    Compose a letter to the patient that is courteous and easy to understand. 
    There should be limited medical jargon and it should be written in layman's language.  
    The letter will briefly summarize the reason for admission, the treatment course, relevant tests and results.  
    It will also summarize any changes to the patient's current medical management including medication changes. 
    Is there any scheduled or recommended follow up?
    If there was a surgery, what are the post-operative or wound care instructions?
    Limit the letter to 3 paragraphs.
    '''.replace('\n', '')
    
    context_str = ''
    df = discharge_sections_df
    cur_df = df[df['hadm_id'] == hadm_id]
    cols = ['HPI']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = f"The history of present illness was: {context_data} \n"
    # cols = ['Medication Lists']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The admission medication list: \n{context_data} \n"
    cols = ['Transitional Issues']
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    context_str = context_str + f"The ongoing issues to be addressed after discharge: \n{context_data} \n"
    # cols = ['BHC']
    # context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n')
    # context_str = context_str + f"The Brief Hospital course was:\n{context_data} \n"
    df = diagnosis_df
    cols = ['icd_title']
    cur_df = df[df['hadm_id'] == hadm_id]
    context_data = cur_df[cols].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).str.cat(sep='\n ')
    context_str = context_str + f"The patient was initially diagnosed with ICD Diagnosis titles: \n{context_data}\n "
    
    
    instruction = f"{topic} \n Instruction: {prompt}"
    context_input = f"<text> Text:{context_str}</text> "
    # print(f"Context for Discharge: {context_input}")
    response = generate_with_model(instruction, context_input)
    discharge_instructions = response
    return discharge_instructions


In [61]:
import pandas as pd

# Lists to hold few-shot learning examples
examples_bhc = []
examples_discharge = []

# Generate examples for few-shot learning
for patient_id in target_df['hadm_id'][:5]:
    # Generate context
    brief_hospital_course_context = generate_bhc(patient_id)  # This function needs to return the context string
    discharge_instructions_context = generate_discharge_instructions(patient_id)  # This function needs to return the context string

    # Retrieve the actual summary from target_df
    bhc_summary = target_df[target_df.hadm_id == patient_id]['brief_hospital_course'].iloc[0]
    discharge_summary = target_df[target_df.hadm_id == patient_id]['discharge_instructions'].iloc[0]

    # Append to the example lists
    examples_bhc.append({
        "context": f"<text>{brief_hospital_course_context}</text>",
        "response": bhc_summary
    })
    examples_discharge.append({
        "context": f"<text>{discharge_instructions_context}</text>",
        "response": discharge_summary
    })


In [21]:
examples_bhc

[{'context': "<text>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501\n\n### Topic:\n    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR)     from a patient's hospital admission.     You will be provided with text and data from a specific patient's medical records and clinical notes    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.      Answer clearly and concisely.     \n Instruction:     Using the following patient specific data,     briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.    \n\n### Input:\n<text> Text:The chief complaint was - Shortness of Breath \nThe History of present illness: \nMs. ___ is a ___ female with history of \nCOPD on home O2, atrial fibrillation on apixaban, hyp

In [11]:
target_df.head()

,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count
0,10001884-DS-35,24962904,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",Ms. ___ is a ___ female with history of \nCOPD...,87,358
1,10003019-DS-22,22774359,"Dear Mr. ___,\n\nIt has been our pleasure to b...",___ male with h/o Hodgkin's lymphoma C1D17 ABV...,140,78
2,10003299-DS-7,29323205,Dear ___ were hospitalized due to symptoms of ...,___ RH female with a PMHx of paramedian pontin...,242,386
3,10003502-DS-7,20459702,It was a pleasure caring for you at ___ \n___....,Hospitalization Summary: \nMs. ___ is an ___ y...,74,50
4,10004322-DS-21,28755331,"Dear Mr. ___,\n\nIt was our pleasure to care f...",___ with h/o psychosis admitted because of mul...,69,287


In [27]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

# Initialize the LLM object with the specified model
model = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B')

INFO 05-10 04:58:37 llm_engine.py:98] Initializing an LLM engine (v0.4.1) with config: model='meta-llama/Meta-Llama-3-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OutOfMemoryError: CUDA out of memory. Tried to allocate 112.00 MiB. GPU 0 has a total capacity of 21.99 GiB of which 51.00 MiB is free. Including non-PyTorch memory, this process has 21.92 GiB memory in use. Of the allocated memory 20.39 GiB is allocated by PyTorch, and 90.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [62]:
def few_shot_prompt(examples, context_input):
    # Add examples to the prompt
    prompt = "\n".join([f"\n### Example {i+1}:\nContext: {ex['context']}\n---\nResponse: {ex['response']}" for i, ex in enumerate(examples)])
    
    prompt += f"\n\n### Actual Task:\nContext: {context_input}\n---\nResponse:"
    
    return prompt

# Function to generate output using few-shot prompt
def generate_with_few_shot_model(examples, context_input, model):
    prompt = few_shot_prompt(examples, context_input)
    # Generate the response using the model
    context_length = len(tokenizer(prompt, return_tensors="pt").input_ids[0])
    max_input_length = min(2048, tokenizer.model_max_length - context_length)
    sampling_params = SamplingParams(temperature=0.3, top_p=0.95, max_tokens=max_input_length)

    outputs = model.generate(prompt, sampling_params)
    response_text = outputs[0].outputs[0].text  # Adjust based on actual output structure
    print(response_text)

In [63]:
# Generate BHC with few-shot learning
def generate_bhc_few_shot(hadm_id, model):
    # Generate context based on hadm_id
    brief_hospital_course_context = generate_bhc(hadm_id)
    return generate_with_few_shot_model(examples_bhc, brief_hospital_course_context, model)

# Generate Discharge Instructions with few-shot learning
def generate_discharge_instructions_few_shot(hadm_id):
    # Generate context based on hadm_id
    context_input = f"<text>Your generated context for {hadm_id}</text>"
    instruction = 'Your instruction for discharge'
    return generate_with_few_shot_model(examples_discharge, instruction, context_input, model)

In [ ]:
generate_bhc_few_shot(28755331, model)

Processed prompts:   0%|                                  | 0/1 [00:00<?, ?it/s]

In [47]:
print(target_df[target_df.hadm_id==20459702]['brief_hospital_course'][3])

Hospitalization Summary: 
Ms. ___ is an ___ yo ___ F with atrial 
fibrillation/flutter on dabigatran, coronary artery disease, 
diastolic heart failure (LVEF 60% in ___, 
hypertension/hyperlipidemia, and vascular dementia who presented 
with nausea and headache and was found to be in acute-on-chronic 
diastolic heart failure in the setting of NSTEMI.


In [60]:
target_df.head()

,note_id,hadm_id,discharge_instructions,brief_hospital_course,discharge_instructions_word_count,brief_hospital_course_word_count
0,10001884-DS-35,24962904,"Dear Ms. ___,\n\nYou were admitted to ___ afte...",Ms. ___ is a ___ female with history of \nCOPD...,87,358
1,10003019-DS-22,22774359,"Dear Mr. ___,\n\nIt has been our pleasure to b...",___ male with h/o Hodgkin's lymphoma C1D17 ABV...,140,78
2,10003299-DS-7,29323205,Dear ___ were hospitalized due to symptoms of ...,___ RH female with a PMHx of paramedian pontin...,242,386
3,10003502-DS-7,20459702,It was a pleasure caring for you at ___ \n___....,Hospitalization Summary: \nMs. ___ is an ___ y...,74,50
4,10004322-DS-21,28755331,"Dear Mr. ___,\n\nIt was our pleasure to care f...",___ with h/o psychosis admitted because of mul...,69,287


In [5]:
for id in target_df['hadm_id'][:5]:
    print(id)

24962904
22774359
29323205
20459702
28755331


In [ ]:
# Example data for few-shot learning
examples_bhc = [
    {
        "context": 
'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501

### Instruction:
    The topic is about clinical notes, medical records, and other text documents from electronic health records (EHR)     from a patient's hospital admission.     You will be provided with text and data from a specific patient's medical records and clinical notes    delimited by xml tags <text> </text>. Use only this data to answer questions about the EHR.      Answer clearly and concisely.     
 Instruction:     Using the following patient specific data,     briefly summarize the major medical conditions, reason for hospital admission, diagnostic workup, and treatment course.    

### Input:
<text> Text:The chief complaint was - Shortness of Breath 
The History of present illness: 
Ms. ___ is a ___ female with history of 
COPD on home O2, atrial fibrillation on apixaban, hypertension, 
CAD, and hyperlipidemia who presents with shortness of breath, 
cough, and wheezing for one day.

The patient reports shortness of breath, increased cough 
productive of ___ red-flected sputum, and wheezing since 
yesterday evening.  She has been using albuterol IH more 
frequently (___) with ipratropium nebs every 4 hours with 
minimal relief. She had to increase her O2 flow up to 4L without 
significant improvement. She was currently taking 10mg of 
prednisone. She has also been taking tiotropium IH, 
theophylline, advair IH at home as prescribed. She denies sick 
contacts. She quit smoking approximately 1 month ago.

She reports an episode of chest pain in waiting room while 
sitting down, non-exertional, resolved after 2 minutes. She 
denies fever/chills, abdominal pain, nausea/vomiting, 
palpitations, and diaphoresis.  

She was recently admitted from ___ to ___ for dyspnea that 
was thought to be secondary to steroid taper for recent COPD 
exacerbation with a component of anxiety (not an acute COPD 
exacerbation) and was treated with steroids and duonebs but no 
antibiotics. She had a CT that showed emphysema but no evidence 
of infection such as ___. Pulmonary was consulted and 
recommended increasing her Advair dose to 500/50 (which was 
done) and switching from theophylline to 
roflumilast and initiation of long-term azithromycin therapy 
(which was deferred for outpatient follow-up) She was initiated 
on a steroid 
taper on ___ of prednisone 30 mg for 3 days, then 20 mg for 3 
days, then 10 mg until outpatient follow-up.

In the ED, initial vital signs were: 97.6 67 132/82 22 97% 4L. 
Exam was notable for limited air movement with wheezing 
bilaterally. Labs were notable for WBC 7.1, H/H 12.8/41.1, Plt 
233, Na 133, K 3.6, BUN/Cr ___, trop < 0.01, BNP 181, lactate 
1.5, VBG 7.43/___. Imaging with CXR showed mild basilar 
atelectasis without definite focal consolidation. The patient 
was given Duonebs and solumedrol 125mg IV. Vitals prior to 
transfer were:

Upon arrival to the floor, she reports her breathing is 
improved.

REVIEW OF SYSTEMS: Per HPI. Denies headache, visual changes, 
pharyngitis, rhinorrhea, nasal congestion, fevers, chills, 
sweats, weight loss, abdominal pain, nausea, vomiting, diarrhea, 
constipation, hematochezia, dysuria, rash, paresthesias, and 
weakness. 
The major procedures were:
N/A 
The patient was diagnosed with:
PRIMARY:
COPD Exacerbation

SECONDARY:
Afib
Anxiety
HTN
CAD 
The patient was initially diagnosed with ICD Diagnosis titles: 
Chronic obstructive pulmonary disease w (acute) exacerbation
 </text> 

### Response:''',
        "response": '''
        Ms. ___ is a ___ female with history of \nCOPD on home O2, atrial fibrillation on apixaban, 
        hypertension, \nCAD, and hyperlipidemia who presents with shortness of breath, \ncough, and wheezing for one day. 
        Pt recently DC'd from hospital \nfor dyspnea, treated only w/nebs and steroids as not thought ___ \ntrue COPD exacerbation, 
        c/f anxiety component. Pt re-admitted \nw/similar Sx, thought ___ COPD exacerbation, received nebs, \nsteroids, azithromycin. 
        Pt's wheezing, cough, SOB improved \nshortly after admission, O2 titrated down & satting well on 2L \nin mid-90s which is baseline. 
        Evaluated by ___, recommended DC to \npulmonary rehab, pt was agreeable. \n\nACTIVE ISSUES\n=================\n# Shortness of Breath:
        Patient with history of COPD and recent \nadmission for dyspnea in the setting of steroid taper. Her \nsymptoms on presentation were 
        consistent with severe COPD given \ndiffuse wheezing and poor air movement. She likely had an \nexacerbation in the setting of a 
        decrease in her steroids. There \nmay also be a component of anxiety. She underwent CT last \nadmission that was negative for 
        infections such as ___. She was \ncontinued on home spiriva, theophylline, advair. She was started \non standing duonebs q6h and 
        albuterol q2h prn and prednisone was \nstarted at 40mg daily with slow taper. She was also given \nazithromycin to complete 5 day 
        course. She had improvement in \nher wheezing and returned to baseline O2 requirement after 48 \nhours. She was seen by ___ who felt 
        that she would benefit from \ndischarge to inpatient pulmonary rehabilitation program. On DC \nto ___ rehab, recommended continued 
        Prendisone 40mg \ndaily for 1x week with slow taper by 5mg every 5 days. ___ also \nconsider starting bactrim ppx with extended 
        duration of steroids \nif unable to wean less than 20mg qd. Will also f/u as outpatient \nwith pulm.\n\nCHRONIC ISSUES: 
        \n==================\n# Anxiety/Insomnia: Continued home lorazepam. Consider starting \nSRRI as an outpatient. \n# Atrial 
        Fibrillation: Continued dilt for rate control and \napixaban for anticoagulation.\n# Hypertension: Continued home imdur, 
        hydrochlorothiazide, and \ndiltiazem.\n# CAD: Cardiac catheterization in ___ without evidence of \nsignificant stenosis 
        of coronaries. ECHO on ___ with EF > \n55% and no regional or global wall motion abnormalities. \nContinued home aspirin 
        and atorvastatin.\n# Anemia: Continued home iron supplements.\n\nTRANSITIONAL ISSUES:\n==========================\n[] For 
        pt's continued COPD exacerbations, recommend finishing 5d \ncourse of Azithromycin, 250mg qd until ___\n[] Recommend extended 
        prednisone taper for pt, 5d 40mg \nPrednisone (to finish ___ followed by 10mg taper every 5 days \n(35mg from ___, 30mg ___, etc...). 
        \n[] Would consider PCP prophylaxis with ___ if unable to wean \nprednisone to less than 20mg daily. \n[] Pt's SOB may have an anxiety 
        component, may benefit from \nstarting SSRI in addition to home benzos already prescribed\n\n# CONTACT: Full Code\n# CODE STATUS: ___ 
        (husband/HCP) ___\n \n___ on Admission:\nThe Preadmission Medication list is accurate and complete.\n1. Acetaminophen 325 mg PO Q4H:PRN
        Pain \n2. Apixaban 5 mg PO BID \n3. Aspirin 81 mg PO DAILY \n4. Atorvastatin 10 mg PO QPM \n5. Diltiazem Extended-Release 240 mg PO
        BID \n6. Docusate Sodium 100 mg PO BID \n7. Dorzolamide 2% Ophth. Soln. 1 DROP BOTH EYES BID \n8. Ferrous Sulfate 325 mg PO DAILY 
        \n9. Fluticasone Propionate NASAL 2 SPRY NU DAILY:PRN allergies \n10. Hydrochlorothiazide 50 mg PO DAILY \n11. Isosorbide 
        Mononitrate (Extended Release) 240 mg PO DAILY \n12. Latanoprost 0.005% Ophth. Soln. 1 DROP BOTH EYES QHS \n13. Multivitamins 
        1 TAB PO DAILY \n14. PredniSONE 10 mg PO DAILY \n15. Ranitidine 300 mg PO DAILY \n16. Theophylline SR 300 mg PO BID \n17. 
        Tiotropium Bromide 1 CAP IH DAILY \n18. Ipratropium Bromide Neb 1 NEB IH Q6H:PRN Wheezing \n19. cod liver oil 1 capsule  
        oral BID \n20. Calcitrate-Vitamin D (calcium citrate-vitamin D3) 315 mg - \n200 units  oral DAILY \n21. albuterol sulfate 
        90 mcg/actuation inhalation Q4H \n22. Fluticasone-Salmeterol Diskus (500/50)  1 INH IH BID \n23. Lorazepam 0.5 mg PO Q8H:PRN 
        Anxiety \n24. Guaifenesin ___ mL PO Q4H:PRN cough
        '''
    },
    {
        "context": "<text>Context 2</text>",
        "response": "Response 2"
    }
]
